<a href="https://colab.research.google.com/github/Eleniy1/LDA_Icelandic/blob/main/Gensim_LDA_model_is.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
s = open("all_stop_words.txt")
stop_words = [line.strip() for line in s]
print(stop_words)

In [ ]:
import re
import numpy as np
from pprint import pprint

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
import spacy

In [ ]:
!pip install pyLDAvis

In [ ]:
! pip install gensim

In [ ]:
import pyLDAvis
import matplotlib.pyplot as plt

In [ ]:
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
df = pd.read_csv('poems_is_tagged.csv')
print(df.lemmat_text)
df.head()

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 
data_words = list(sent_to_words(df.lemmat_text))
print(data_words[:10])

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)

In [ ]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [ ]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

In [ ]:
id2word[0]

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model.print_topics()

[(0,
  '0.012*"kenna" + 0.012*"haust" + 0.011*"heiður" + 0.011*"jafnvel" + 0.011*"lausnari" + 0.010*"svoddan" + 0.009*"miskunn" + 0.009*"gull" + 0.008*"lina" + 0.007*"sæla"'),
 (1,
  '0.031*"jesu" + 0.024*"burt" + 0.020*"fljuga" + 0.015*"ver" + 0.013*"blar" + 0.008*"frjals" + 0.006*"fastur" + 0.006*"þoka" + 0.006*"ræða" + 0.006*"hestur"'),
 (2,
  '0.009*"tyndur" + 0.008*"pæla" + 0.008*"kristni" + 0.006*"sprettur" + 0.006*"snura" + 0.005*"sorry" + 0.005*"heiðingi" + 0.005*"mur" + 0.005*"bugur" + 0.005*"hrina"'),
 (3,
  '0.018*"hugsun" + 0.018*"þogn" + 0.018*"grænn" + 0.016*"loksins" + 0.012*"ga" + 0.010*"skjol" + 0.009*"fri" + 0.007*"gulur" + 0.007*"heimskur" + 0.006*"viða"'),
 (4,
  '0.043*"guð" + 0.017*"herra" + 0.016*"barn" + 0.014*"skulu" + 0.012*"vinna" + 0.011*"kunna" + 0.011*"land" + 0.010*"kona" + 0.009*"kross" + 0.009*"hold"'),
 (5,
  '0.023*"skina" + 0.014*"vin" + 0.013*"sætur" + 0.013*"gleðja" + 0.012*"eyða" + 0.012*"stoll" + 0.012*"þakka" + 0.010*"petur" + 0.008*"kvelja" + 0

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
! wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [ ]:
!unzip mallet-2.0.8.zip

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=16, id2word=id2word)

In [ ]:
pprint(ldamallet.show_topics(formatted=False))
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, text, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics
    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_words_bigrams, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_bigrams, start=2, limit=40, step=6)

In [ ]:
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
pprint(ldamallet.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words_bigrams):
    sent_topics_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data_words_bigrams)
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)
df_dominant_topic.Dominant_Topic

In [ ]:
#!pip uninstall pandas
!pip install pandas==1.1.5
import pandas as pd

In [ ]:
df_dominant_topic.to_csv("icelandic_topics.csv", index=False)

In [ ]:
df2 = pd.read_csv("icelandic_topics.csv")
df2